**PLEASE NOTE:** All data contained in this and subsequent notebooks are synthetic data that have been derived from a real-world dataset.  The datasets used for this notebooks contain no [Protected health information (PHI)](https://en.wikipedia.org/wiki/Protected_health_information) and are used for the purposes of demonstrating the machine learning development lifecycle used for this analysis.

# Introduction




### Home Healthcare Industry

Home health is a nursing specialty in which nurses provide multidimensional home care to patients of all ages. Home health care is a cost efficient way to deliver quality care in the convenience of the client's home. 
 Home health nurses create care plans to achieve goals based on the client's diagnosis. These plans can include preventive, therapeutic, and rehabilitative actions. Home health nurses also supervise certified nursing assistants. The professional nursing organization for home health nurses is the Home Healthcare Nurses Association (HHNA). Home health care is intended for clients that are well enough to be discharged home, but still require skilled nursing personnel to assess, initiate and oversee nursing interventions.

From Wikipedia: [Home Health Nursing](https://en.wikipedia.org/wiki/Home_health_nursing)

### Business Challenge

The primary focus of the home healthcare provider is to keep their patients healthy.  Home healthcare nurses make at-home visits, monitor a patients health, and take measures to proactively address health issues that might require a patient's hospitalization.  Beyond individual patient health, there are financial incentives from insurers for home healhcare organization that maintain the patient population under their care healthy and outside of the hospital.

### Data Description

During at-home visits by home healthcare nurses, patient health statistics were collected on a recurring basis with the frequency varying across the patient population.  

The following datasets and corresponding data-points were collected: 
<br/><br/>

- **Patients**
  - Age
  - Gender  
* **Visits**
  * Date of Visit (i.e., year, month, week number, date)
  * Total Visits - Cumulative number of visits to-date.
  * Total Hospitalizations for Week - Cumulative number of hospitalizations on a particular week if any.
  * Total Hospitalizations - Cumulative number of hospitalizations to-date if any.

* **Health Statistics**
  * 150+ Health statistics including blood pressure, pulse, weight and medications taken by a particular patient.

### Analysis and Model Deployment Goals

In order to proactively identify, monitor and predict the likelihood of a patient requiring hospitalization a classification machine learning model was developed.  

The primary goals for development were to:


1. Develop a machine learning model that can aid in identifying patients at highest risk of hospitalizaiton on a weekly basis.
2. Understand the risk factors that contribute to a high risk of hospitalization.
3. Deploy the model and refresh with patient visit, health statistic, and hospitalization risk data on an on-going basis.



# Initialize Notebook

### Import Packages

In [3]:
import hashlib as hl
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering

from ydata_profiling import ProfileReport

import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

### Load Utilities

Load common utility functions used throughout the data cleansing and transformation steps.

In [4]:
%run ./utils/utils_diagnosis.ipynb

# Load Source Data


## Patients

Load patient data containing age and gender information. (Additional features per patient are available such as demographic data.  For the purposes of this demo analysis only the age feature is being used.)

In [5]:
patients_df = pd.read_csv('./data/diagnosis_patients.csv')  

## Visits

Load patient visit history. Each record indicates a home visit by a patient caretaker during which health stats were collected or a hospitalization was recorded.

In [6]:
visits_df = pd.read_csv('./data/diagnosis_visits.csv')  

## Health Statistics

Patient health statistics captured during visits.

In [7]:
health_stats_df = pd.read_csv('./data/diagnosis_health_stats.csv') 

# Data Profiling

The main goal with all subsequent data transformation steps are to ensure we have as complete and informative dataset as possible for prediction and health profiling of the patient population.

We'll begin with some basic data profiling to understand the datasets we're working with and proceed from there. All data profiles are saved to the ./html/ directory.

### Profile | Patients

Each row of patient data consists of a unique id (p_uid), age, and gender for a patient:

In [8]:
patients_df.head(5)

,uid,age,gender
0,003161fb221a48c334946bd91861e6dce983beda77327b...,69.00,female
1,0033dc924161304ec1cbecf867d9b881c4c9a124d5e4b1...,67.10,female
2,00b9a66b99c26fc102bc2cf7d7790b48ac8919d2d14203...,56.00,male
3,014dd96fbf7f2fbaba7e9a1abbc7243bb5567e95195c6e...,70.11,male
4,0202192e872ce848bd5c8cfe05da15d39132819a34f782...,64.90,female


Each row should be unique combination of p_uid, age and gender. Let's verify:

In [9]:
len(patients_df) == len(patients_df.drop_duplicates())

True

Age should be cast to int.

In [10]:
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1635 entries, 0 to 1634
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   uid     1635 non-null   object 
 1   age     1635 non-null   float64
 2   gender  1635 non-null   object 
dtypes: float64(1), object(2)
memory usage: 38.4+ KB


In [11]:
patients_df.age = patients_df.age.astype(int)

#### Summary | Patient Data Profiling

From our profiling of patient data, there are a few key points:

* There are a total of 1,154 patients
* 1,154 are female and 481 are male
* The minimum age is 23 and maximum age is 103.
* The average age is ~65 years old.
* Age shows a nearly symetrical distribution with a skewness of -.08

In [49]:
patients_report = ProfileReport(patients_df.drop('uid', axis=1), title='Original Data | Patients')
patients_report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

### Profile | Visits

Each row of patient visit data consists of a unique patient identifier, visit year, month, week number and visit week date.  There are also fields capturing the cumulative number of visits, hospitalizations in a particular week, and total historical hospitalization count.

In [13]:
visits_df.head(5)

,uid,visit_date_year,visit_date_month,visit_date_week_no,visit_date_week_dt,ttl_visits,ttl_hosp_week,ttl_hosp_count,hospitalized
0,713be175a617a8ed4912693cf7276d5bdfddc3866eae65...,2018,8,34,2018-08-20,0,0,0,0
1,713be175a617a8ed4912693cf7276d5bdfddc3866eae65...,2018,8,35,2018-08-27,5,0,0,0
2,713be175a617a8ed4912693cf7276d5bdfddc3866eae65...,2018,9,35,2018-08-27,7,0,0,0
3,713be175a617a8ed4912693cf7276d5bdfddc3866eae65...,2018,9,36,2018-09-03,14,0,0,0
4,713be175a617a8ed4912693cf7276d5bdfddc3866eae65...,2018,9,37,2018-09-10,21,0,0,0


Each row in the visits data set should correspond to a unique combination of fields.  Let's verify:

In [14]:
len(visits_df) == len(visits_df.drop_duplicates())

True

Let's verify our variables are of the correct data types:

In [15]:
visits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28215 entries, 0 to 28214
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   uid                 28215 non-null  object
 1   visit_date_year     28215 non-null  int64 
 2   visit_date_month    28215 non-null  int64 
 3   visit_date_week_no  28215 non-null  int64 
 4   visit_date_week_dt  28215 non-null  object
 5   ttl_visits          28215 non-null  int64 
 6   ttl_hosp_week       28215 non-null  int64 
 7   ttl_hosp_count      28215 non-null  int64 
 8   hospitalized        28215 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 1.9+ MB


Cast variables to correct types:

In [16]:
# Remove duplicate rows
visits_df.drop_duplicates(inplace=True)
    
# Convert year, month, and week_no to ints for now
visits_df.visit_date_year = visits_df.visit_date_year.astype(int)
visits_df.visit_date_month = visits_df.visit_date_month.astype(int)
visits_df.visit_date_week_no = visits_df.visit_date_week_no.astype(int)

# Convert 'visit_date_week_dt' to datetime
visits_df.visit_date_week_dt = pd.to_datetime(visits_df.visit_date_week_dt)

# Convert selected columns to integers
visits_df[['ttl_visits', 'ttl_hosp_week', 'ttl_hosp_count', 'hospitalized']] = \
visits_df[['ttl_visits', 'ttl_hosp_week', 'ttl_hosp_count', 'hospitalized']].astype(int)

All variables now have the correct data type:

In [17]:
visits_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28215 entries, 0 to 28214
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   uid                 28215 non-null  object        
 1   visit_date_year     28215 non-null  int32         
 2   visit_date_month    28215 non-null  int32         
 3   visit_date_week_no  28215 non-null  int32         
 4   visit_date_week_dt  28215 non-null  datetime64[ns]
 5   ttl_visits          28215 non-null  int32         
 6   ttl_hosp_week       28215 non-null  int32         
 7   ttl_hosp_count      28215 non-null  int32         
 8   hospitalized        28215 non-null  int32         
dtypes: datetime64[ns](1), int32(7), object(1)
memory usage: 1.4+ MB


#### Summary | Visit Data Profiling

From our profiling of patient data, there are a few key points:

* The majority of visits occured in 2020 (10,489), then 2019 (8,974) and 2021 (4,774). 
* The Majority of visits (92%) were non-hospitalization visits (i.e., 0 hosptilizations for the week) meaning, our dataset is highly imbalanced in terms of our target variable--visits with hospitalizations.

In [48]:
visits_report = ProfileReport(visits_df.drop('uid', axis=1), title='Original Data | Visits')
visits_report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

### Profile | Health Stats

The health stats data **should** be unique at the row level with each row corresponding to a visit for a specific patient (uid).  However, due to the manner in which the data was captured, the field containing patient medications (med_profile_medication) will cause a specific visit's information to be duplicated for each medication that was recorded for a patient. For example, if on a particular visit a patient was taking 5 medications, there would be 5 rows corresponding to each medication with all other fields duplicated for each row.  Ultimately, we'd like to predict each patient's likelihood of hospitalization on a weekly basis using health stats and visit history data.  So, we'll need to align the grain of both datasets to the weekly level.

In [19]:
health_stats_df.head(5)

,uid,visit_date_week_dt,med_profile_medication,visit_date,visit_date_year,visit_date_month,visit_date_week_no,bp_sys,bp_dia,weight_lbs,...,neuro_unsteady_gait,neuro_paralysis,neuro_tremors,psychological_behavioral,documentation_type,bp_type,temp_t,device_o,pulse_r,resp_descr
0,f4b99bb2d02ee6c067dc6cbe54f2aa394f81aafb6bafb6...,2020-03-09,BENADRYL,2020-03-12,2020,3,11,123.020323,71.974086,146.0,...,Yes,No,No,Asymptomatic,Supervisory Visit,sitting,Tympanic,no device,Reg,reg
1,f4b99bb2d02ee6c067dc6cbe54f2aa394f81aafb6bafb6...,2020-03-09,BENADRYL,2020-03-12,2020,3,11,123.020323,71.974086,146.0,...,Yes,No,No,Asymptomatic,Supervisory Visit,sitting,Tympanic,no device,Reg,reg
2,f4b99bb2d02ee6c067dc6cbe54f2aa394f81aafb6bafb6...,2020-03-09,BENADRYL,2020-03-12,2020,3,11,123.020323,71.974086,146.0,...,Yes,No,No,Asymptomatic,Supervisory Visit,sitting,Tympanic,no device,Reg,reg
3,f4b99bb2d02ee6c067dc6cbe54f2aa394f81aafb6bafb6...,2020-03-09,BENADRYL,2020-03-12,2020,3,11,123.020323,71.974086,146.0,...,Yes,No,No,Asymptomatic,Supervisory Visit,sitting,Tympanic,no device,Reg,reg
4,f4b99bb2d02ee6c067dc6cbe54f2aa394f81aafb6bafb6...,2020-03-09,BENADRYL,2020-03-12,2020,3,11,123.020323,71.974086,146.0,...,Yes,No,No,Asymptomatic,Supervisory Visit,sitting,Tympanic,no device,Reg,reg


In [20]:
health_stats_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504896 entries, 0 to 504895
Data columns (total 172 columns):
 #    Column                                              Non-Null Count   Dtype  
---   ------                                              --------------   -----  
 0    uid                                                 504896 non-null  object 
 1    visit_date_week_dt                                  504896 non-null  object 
 2    med_profile_medication                              503309 non-null  object 
 3    visit_date                                          504896 non-null  object 
 4    visit_date_year                                     504896 non-null  int64  
 5    visit_date_month                                    504896 non-null  int64  
 6    visit_date_week_no                                  504896 non-null  int64  
 7    bp_sys                                              389144 non-null  float64
 8    bp_dia                                              

Cast variables to correct types:

In [21]:
# The year, month, and week_no variables should be strings
health_stats_df.visit_date_year = health_stats_df.visit_date_year.astype(int)
health_stats_df.visit_date_month = health_stats_df.visit_date_month.astype(int)
health_stats_df.visit_date_week_no = health_stats_df.visit_date_week_no.astype(int)

# The visit date week field should be a date field
health_stats_df.visit_date_week_dt = pd.to_datetime(health_stats_df.visit_date_week_dt)

# The visit date week field should be a date field
health_stats_df.visit_date = pd.to_datetime(health_stats_df.visit_date)

Each combination of uid and visitdate should guarantee uniqueness.  We see that their exist duplicates in our dataset:

In [22]:
len(health_stats_df) == len(health_stats_df[['uid','visit_date']].drop_duplicates())

False

As mentioned previously, due to the manner in which the data is collected, the med_profile_medication causes duplication.  Let's drop the med_profile_medication field for now to check for any other duplication at the visit level. (We will one-hot encode these values and merge it back into the health stats dataset in subsequent steps)

In [23]:
# Drop medication column to encode add back later
health_stats_meds_df = health_stats_df[['med_profile_medication']]
health_stats_df.drop('med_profile_medication', axis=1, inplace=True)

After analyzing the records, we determine it's safe to drop the duplicate rows and simply keep the last set of values.

In [24]:
health_stats_df.drop_duplicates(subset=['uid','visit_date'], keep='last', inplace=True)

Once we drop the med_profile_medication field the dataframe now has no duplicates.

In [25]:
len(health_stats_df) == len(health_stats_df[['uid','visit_date']].drop_duplicates())

True

The health statistics dataframe has now been deduped for all fields with the exception of medications (we will deal with that field later)

#### Summary | Health Statistic | Meds | Data Profiling

From the profiling of Meds data, we can see the most prevalent type of medications: 

**1. Atorvastatin Calcium** - Used together with a proper diet to lower cholesterol and triglyceride (fats) levels in the blood.<br>
**2. Aspirin** - Can treat mild to moderate pain, inflammation or arthritis. It also lowers your risk of heart attack, stroke or blood clot.<br> 
**3. Amlodipine Besylate** - Used to treat high cholestorel.

Once we've joined this data to our visit data we can begin to determine correlation with our target variable--hospitalizations.


In [51]:
health_stats_meds_report = ProfileReport(health_stats_meds_df, title='Health Statistics | Medications')
health_stats_meds_report.to_file("./htm/health_stats_meds_report.html")

#### Summary | Health Statistic Data Profiling

There are over 171 health statistics (outside of medications taken) as categorical variables within our dataset. The majority of variables are highly imbalanced, so we will need to determine which of these features are correlated with hospitalizations and which do not provide signficant predictive power and can be discarded.

In [27]:
health_stats_non_meds_report = ProfileReport(health_stats_df, title='Health Statistics | Non-Medications')
health_stats_non_meds_report

We now move on to imputation of missing values.

# Health Statistic Imputation Strategy

As a reminder, the dataset we're working with consists of at-home healthcare visits. There are some visits during which health statistics we're captured.  However, during the majority of visits no statistics were collected.  Key to our imputation strategy is to first forward-fill and then back-fill health statistics data for each patient across their entire visit history.  For cases in which there is a statistic that was never captured for a patient, we will impute using the mean for specific numeric statistic (e.g., systolic blood pressure)  and the most frequent occurence for a categorical statistic (e.g., nausea, medications taken etc.)


At a high level, the main imputation steps are:
<br></br>

1. Propagate health statistics forward and backward across each patient's visit history.
2. For cases in which a specific health statistic was not entered for a patient, impute the average values from the entire patient population for numeric features or the most frequently occuring value for categorical features.
3. Aggregate data at weekly granularity for hospitalization prediction on a weekly basis.
4. Union all visits with visits where health statistics were captured. 
5. Propagate health statistics forward and backward across each patient's visit history.

![imputation_strategy](img/diagnosis_imputation_strategy_med_v3.png)

## Propagate health statistics forward and backward across each patient's visit history.

Merge patient and health stats data.

In [28]:
# Merge patient and diagnosis data
patients_health_stats_df = patients_df.merge(health_stats_df, left_on='uid', right_on='uid', how='inner')

One-hot encode categorical variables that were initally dropped from the health statistics dataframe and standardize column names (i.e., remove punction, convert to lowercase and replace spaces for underscores)

In [29]:
# Dummy encode meds
health_stats_meds_df = pd.get_dummies(health_stats_meds_df, prefix='med_')
health_stats_meds_df = standardize_column_names(health_stats_meds_df)

Rejoin dummy encoded medication variables to rest of health stastic data

In [30]:
# Rejoin patient health stats with meds one-hot encoded dummy values
patients_health_stats_df = patients_health_stats_df.join(health_stats_meds_df)

Now that we've dummy encoded the medication features, we see that the number of columns in our dataset has balloneed to 1,625.  We will also need to determine which medications are correlated with hospitalizations to determine which we will keep and which we can drop from our dataset.

In [31]:
len(patients_health_stats_df.columns)

1625

In [32]:
null_counts = get_null_counts(patients_health_stats_df)
print('There are '+f"{null_counts:,}"+ ' remaining null values.') 

There are 114,732 remaining null values.


### Impute Missing Values

#### Impute Missing Values by Patient

Not all health statistics were captured during each patient visit (e.g., weight may have been take on the initial visit for a particular patient but not for a subsequent visit).  Therefore, we will impute missing statistics per patient first based on the last known statistic captured (i.e., forward fill the data-point) and then impute remaining missing values by back-filling missing data points. The "impute_missing_values_by_patient" function does this as described in the section: [Health Statistic Imputation Strategy](#Health-Statistic-Imputation-Strategy)

In [33]:
sort_cols = ['uid', 'visit_date_week_dt']
group_key = 'uid'

# patients_health_stats_df = impute_missing_values_by_group(patients_health_stats_df, 'uid', sort_cols)

patients_health_stats_df = patients_health_stats_df.sort_values(by=sort_cols)

some_cols = patients_health_stats_df.columns.to_list()
some_cols.remove(group_key)

patients_health_stats_df[[group_key] + some_cols] = patients_health_stats_df[[group_key] + some_cols].groupby(patients_health_stats_df[group_key]).ffill().groupby(patients_health_stats_df[group_key]).bfill()
    
patients_health_stats_df.drop_duplicates(inplace=True)

In [34]:
null_counts = get_null_counts(patients_health_stats_df)
print('There are '+f"{null_counts:,}"+ ' remaining null values.')

There are 20,464 remaining null values.


We've reduced the number of null values by patient by imputing statistics captured during a prior or subsequent visit.  Based on the remaining columns with missing values and the percentage of missing values, it is safe to continue imputing the remaining values using the sample mean for numeric values and frequency for categorical.

In [35]:
percentage_of_missing_values(patients_health_stats_df)[1:20]

,column_name,percent_missing
1,paraprofessional_response,11.64
2,change_in_plan_of_care_needed,9.60
3,complies_with_plan_of_care,9.34
4,device_walker,8.98
5,device_cane,5.46
6,patient_caregiver_satisfied_with_service,3.47
7,bp_type,2.68
8,resp_descr,2.61
9,temp_t,2.20
10,lungs_right_posterior_middle,2.03


## Cases for which a specific health statistic was never entered for a patient

#### Impute Remaining Missing Values

Now we will impute the remaining missing values.

In [36]:
# Drop visit date columns
df_non_diagnosis_cols = patients_health_stats_df[['visit_date_year', 'visit_date_month',
       'visit_date_week_no','uid', 'visit_date',
       'visit_date_week_dt']]

patients_health_stats_df.drop(df_non_diagnosis_cols.columns, axis=1, inplace=True)

# Identify numeric and categorical columns for imputation
num_cols = patients_health_stats_df.select_dtypes(include=['float64', 'int32', 'int64']).columns
cat_cols = patients_health_stats_df.select_dtypes(include=['object']).columns

##### Numeric Value Imputation

For remaining missing numeric values, we will impute by taking the mean of the missing statistic across the full patient population.

In [37]:
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
imputer = imputer.fit(patients_health_stats_df[num_cols])
patients_health_stats_df[num_cols] = imputer.transform(patients_health_stats_df[num_cols])

##### Categorical Value Imputation

For categorical value imputation, we will simply impute the constant "no_value" when a data-point is missing.

In [38]:
# Categorical values
imputer = SimpleImputer(strategy='most_frequent')
imputer = imputer.fit(patients_health_stats_df[cat_cols])
patients_health_stats_df[cat_cols] = imputer.transform(patients_health_stats_df[cat_cols])

In [39]:
# Rejoin all imputed columns
patients_health_stats_df = df_non_diagnosis_cols.join(patients_health_stats_df)

In [40]:
null_counts = get_null_counts(patients_health_stats_df)
print('There are '+f"{null_counts:,}"+ ' remaining null values.')

There are 0 remaining null values.


## Aggregate Data at the Weekly Granularity

Aggregate visit and health statistics data at the weekly level.  Visits during which health statistics were gathered for a patient occured mainly at weekly intervals with a few exceptions.  For cases in which more than one visit occured in one week we will take the max values for all statistics.

In [41]:
grouping_cols = ['uid','visit_date_week_dt']

patients_health_stats_by_week_df = patients_health_stats_df.sort_values(by=grouping_cols).groupby(grouping_cols).max().reset_index()

# Drop visit_date since we have aggregated at weekly granularity
patients_health_stats_df.drop('visit_date', axis=1,inplace=True)

In [42]:
len(patients_health_stats_by_week_df)

4177

## Union Visit and Health Statistic Data

In [43]:
visits_health_stats_by_week_df = pd.concat([patients_health_stats_by_week_df, visits_df])

In [44]:
null_counts = get_null_counts(visits_health_stats_by_week_df)
print('There are '+f"{null_counts:,}"+ ' remaining null values.')

There are 91,450,016 remaining null values.


## Propagate health statistics forward and backward across each patient's visit history.

In [45]:
visits_health_stats_by_week_df = visits_health_stats_by_week_df.sort_values(by=sort_cols)

some_cols = visits_health_stats_by_week_df.columns.to_list()
some_cols.remove(group_key)

visits_health_stats_by_week_df[[group_key] + some_cols] = visits_health_stats_by_week_df[[group_key] + some_cols].groupby(visits_health_stats_by_week_df[group_key]).ffill().groupby(visits_health_stats_by_week_df[group_key]).bfill()
    
visits_health_stats_by_week_df.drop_duplicates(inplace=True)

In [46]:
null_counts = get_null_counts(visits_health_stats_by_week_df)
print('There are '+f"{null_counts:,}"+ ' remaining null values.')

There are 0 remaining null values.


# Save Dataset for EDA

In [47]:
visits_health_stats_by_week_df.to_parquet('data/diagnosis_health_stats_hosp_by_week_01.parquet')